In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
from fastai.imports import *
import fastai
import fastai.utils.collect_env
from fastai.widgets import *
fastai.utils.collect_env.show_install(1)



```text
=== Software === 
python       : 3.7.6
fastai       : 1.0.61
fastprogress : 0.2.3
torch        : 1.5.0
torch cuda   : None / is **Not available** 

=== Hardware === 
No GPUs available 

=== Environment === 
platform     : Linux-5.4.104+-x86_64-with-debian-buster-sid
distro       : #1 SMP Tue Apr 6 09:49:56 PDT 2021
conda env    : Unknown
python       : /opt/conda/bin/python3.7
sys.path     : /kaggle/working
/kaggle/lib/kagglegym
/kaggle/lib
/opt/conda/lib/python37.zip
/opt/conda/lib/python3.7
/opt/conda/lib/python3.7/lib-dynload

/root/.local/lib/python3.7/site-packages
/opt/conda/lib/python3.7/site-packages
/src/bq-helper
/opt/conda/lib/python3.7/site-packages/IPython/extensions
/root/.ipython
no supported gpus found on this system
```

Please make sure to include opening/closing ``` when you paste into forums/github to make the reports appear formatted as code sections.

Optional package(s) to enhance the diagnostics can be installed with:
pip install distro
Once installed,

In [ ]:
from fastai import *
from fastai.vision import *
#from fastai.transforms import *
#from fastai.conv_learner import *
#from fastai.model import *
#from fastai.dataset import *
#from fastai.sgdr import *
#from fastai.plots import *

In [8]:
path_img = '/kaggle/input/he-danceforms/dataset/train'
fnames = get_image_files(path_img)
fnames[:5]

In [ ]:
df1 = pd.read_csv('/kaggle/input/he-danceforms/dataset/train.csv', header='infer')
df1.head()

In [ ]:
tfms = get_transforms( max_lighting=0.3, max_warp=0.2, max_zoom=1.2)
test_df = pd.read_csv('/kaggle/input/he-danceforms/dataset/test.csv', header='infer')
test_df.head()
test_images = ImageList.from_df(test_df, path='/kaggle/input/he-danceforms/dataset/test')
test_images[0]

In [ ]:
data = ImageDataBunch.from_df('/kaggle/input/he-danceforms/dataset/train',\
                              df1, ds_tfms=get_transforms(), size=224).normalize(imagenet_stats)#.add_test(test_images)
test = ImageList.from_df(test_df, '/kaggle/input/he-danceforms/dataset/test')
data.add_test(test)
data.show_batch(rows=3, figsize=(7,6))

In [ ]:
print(data.classes)
print(data.c)

In [ ]:
def train_datafn (imagesize,batchsize) : 
    train_img = (ImageList.from_df(df1, path='/kaggle/input/he-danceforms/dataset/train')
        .split_by_rand_pct(0.2,seed =344) ##20% of the data is used as validation
        .label_from_df()
        #.add_test(test_img)
        .transform(tfms = get_transforms(), size=imagesize)
        .databunch(path='.',  device= torch.device('cuda:0')) #bs=batchsize,
        .normalize(imagenet_stats)
       )
    test = ImageList.from_df(test_df, '/kaggle/input/he-danceforms/dataset/test')
    train_img.add_test(test)
    return train_img

In [ ]:
def get_ensemble(nmodels):
    ens_model = [] # Empty List of ensemble model, I will store the trained learner object here 
    learning_rate =[2.95e-04,3e-04,3e-04] # List of learning rate for each model 
    model_list = [models.resnet152,models.resnet50,models.densenet201] ##List of Models . You can add resnet ones in the mix
    for i in range(nmodels):
        print(f'-----Training model: {i+1}--------')
        
        #learn.model_dir = '/kaggle/working'
        if(i == 0 or i == 1 ):            
            print('training for 224x224')
            data = train_datafn(224,64)
            learn_resnet = cnn_learner(data, model_list[i], metrics=[error_rate, accuracy,AUROC()])
            learn_resnet.freeze_to(-1)
            learn_resnet.fit_one_cycle(10)   # using the learning rate assigned for the first model
            learn_resnet.lr_find()
            learn_resnet.recorder.plot(suggestion=True)
            learn_resnet.freeze()
            learn_resnet.fit_one_cycle(20)
        
        
            learn_resnet.save(f'ensem_model_{i}.weights')
            ens_model.append(learn_resnet)
            print(f'-----Training of model {i+1} complete----')
        
        if(i == 2):
            data = train_datafn(64,64)
            learn_resnet = cnn_learner(data, model_list[i], metrics=[error_rate, accuracy,AUROC()])
            learn_resnet.freeze_to(-1)
            print('training for 128x128')
            learn_resnet.set_data = train_datafn(128,64) # Train the model for imagesize 128
            learn_resnet.lr_find()
            learn_resnet.recorder.plot(suggestion=True)
            learn_resnet.fit_one_cycle(10,slice(learning_rate[i])) # using the learning rate for the first model 
        
            print('training for 150x150')
            learn_resnet.set_data = train_datafn(150,96) #Train the model for imagesize 150
            learn_resnet.fit_one_cycle(10,slice(learning_rate[i]))   # using the learning rate assigned for the first model   
        
            print('training for 224x224')
            learn_resnet.set_data = train_datafn(224,96) #Train the model for imagesize 150
            learn_resnet.fit_one_cycle(20,slice(learning_rate[i]))   # using the learning rate assigned for the first model
        
        
            learn_resnet.save(f'ensem_model_{i}.weights')
            ens_model.append(learn_resnet)
            print(f'-----Training of model {i+1} complete----')
            
             
        
    return ens_model

In [ ]:
!pip install "torch==1.4" "torchvision==0.5.0"

In [ ]:
#ens = get_ensemble(1)
data = train_datafn(224,64)
# learn_resnet = cnn_learner(data, models.resnet50, metrics=[error_rate, accuracy,AUROC()])
# learn_resnet.freeze_to(-1)
# learn_resnet.fit_one_cycle(6)   # using the learning rate assigned for the first model
# learn_resnet.save(f'ensem_model_{i}.weights')
# ens_model.append(learn_resnet)
# print('-----Training of model complete----')

In [ ]:
ens  = learn_resnet
ens_test_preds = [] ## Creating a list of predictions 

preds,_ = ens.TTA(ds_type=DatasetType.Test)
print(np.array(preds).shape)
ens_test_preds.append(np.array(preds))

In [ ]:
ens_preds = np.mean(ens_test_preds, axis =0)
print(ens_preds.shape)
print(ens_preds)
labels = np.argmax(ens_preds, 1)
test_df['Class'] = labels
#['bharatanatyam', 'kathak', 'kathakali', 'kuchipudi', 'manipuri', 'mohiniyattam', 'odissi', 'sattriya']
test_df['Class'] = test_df['Class'].map({0: 'bharatanatyam', 1: 'kathak', 2:  'kathakali', 3:  'kuchipudi',4:'manipuri',5:'mohiniyattam',6:'odissi',7:'sattriya'})
test_df.to_csv('submission_ENS_2.csv')